# Flask Basics, Lesson 6!!!

## Flask_Login

This is the 2nd tool for User Authentication!

the *__flask_login__* library is an extremely useful tool for this process!!

![SegmentLocal](https://media0.giphy.com/media/iOjNaLUI4oBgc/giphy.gif "segment")

The best way to show you how to use it is a large example!!:

In this example, app.py will be in a the same directory as myprojects, which is another directory that hold the rest of the files!!

In [ ]:
# __init__.py
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_login import LoginManager

login_manager = LoginManager()

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mysecretkey'
basedir = os.path.abspath(os.path.dirname(__file__))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'+os.path(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
Migrate(app, db)

login_manager.init_app(app)
#Pass your application to the login manager!
login_manager.login_view = 'login'
#the app will have a view called 'login' and that will link this to our login manager 

Now we have to make our __Models!!__ which will include the User

In [ ]:
# models.py
from myproject import db, login_manager
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import UserMixin

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(user_id)
#Allows us to get the specific user to see their ID

class User(db.Model, UserMixin):
    #UserMixin has all the management features of logging in users and authorizing them, so we don't have to do that ourselves 
    
    __tablename__ = 'users'
    
    id = db.Column(db.Integer, primary_key = True)
    email = db.Column(db.String(64), unique = True, index = True)
    #passing 64 characters sets a string limit, so the email cannot be more than 64 characters
    #unique = True, makes sure that two users don't have the same email address
    usernmane = db.Column(db.String(64), unique = True, index = True)
    password_hash = db.Column(db.String(128))
    
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
    

Now we make our __Forms!__ one for logging in and one for registering!

In [ ]:
#forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField 
from wtforms.validators import DataRequired, Email, EqualTo
# The validator Email will make sure what is entered is in email format, such as includes the @
# The validator EqualTo will make sure two things match, such as confirming password when registering 
from wtforms import ValidationError 

class LoginForm(FlaskForm):
    email = StringField('Email', validators = [DataRequired(), Email()])
    password = PasswordField('Password', validators = [DataRequired()])
    submit = SubmitField('Login')

class RegistrationForm(FlaskForm):
    email = StringField('Email', validators = [DataRequired(), Email()])
    username = StringField('Username', validators = [DataRequired()])
    password = PasswordField('Password', validators = [DataRequired(), EqualTo('pass_confirm', message = 'Passwords must match!')])
    # the message is what is presented if they are not equal 
    pass_confirm = PasswordField('Confirm Password', validators = [DataRequired()])
    submit = SubmitField('Register!')
    
    def check_email(self, field):
        if User.query.filter_by(email = field.data).first():
            raise ValidationError('Your email has already been registered!')
        #Checks to see if the email is already taken
    
    def check_username(self, field):
        if User.query.filter_by(username = field.data).first():
            raise ValidationError('Username is taken!')
        #Like the above function but with username

Now we will make our app.py, which contains our view functions!!

In [ ]:
#app.py
from myprojects import app,db
from flask import render_template, redirect, request, url_for, flash, abort
from flask_login import login_user, login_required, logout_user
from myproject.models import users
from myproject.forms import LoginForm, RegistrationForm 

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/welcome')
@login_required
#including this means that a user has to be logged in to see this page
def welcome_user():
    return render_template('welcome_user.html')

@app.route('/logout')
@login_required
#must be logged in to logout!
def logout():
    logout_user()
    flash("You logged out!")
    return redirect(url_for('home'))

@app.route('/login', methods = ['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(email = form.email.data).first()
        
        if user.check_password(form.password.data) and user is not none:
            login_user(user)
            flash('Logged in successfully!!')
            
            next = request.args.get('next')
            
            if next == None or notnext[0] == '/':
                next = url_for('welcome_user')
                
            return redirect(next)
        
    return render_template('login.html', form=form)

@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm():
        user = User(email = form.email.data, username = form.username.data, password = form.password.data)
        db.session.add(user)
        db.session.commit()
        flash("Thanks for registering!")
        return redirect(url_for('login'))
    return render_template('register.html', form=form)

if __name__ == '__main__':
    app.run(debug = True)

Now we want to make our html files, starting with base:

In [ ]:
# base.html
<!DOCTYPE html>
<html lang = "en" dir = "ltr">
    <head>
        bootstrap links!!!
        <meta charset = "utf-8">
        <title></title>
    </head>
    <body>
        <ul class = "nav">
            <li class = 'nav-link'<a href = "{{url_for('home')}}">Home</a></li>
            {% if current_user.is_authenticated %}
            <li class = 'nav-link'><a href = "{{url_for('logout')}}">Logout</a></li>
            {% else %}
            <li class = 'nav-link'><a href = "{{url_for('login')}}">Login</a></li>
            <li class = 'nav-link'><a href "{{url_for('register')}}">Register</a></li>
            {% endif %}
        </ul>
        {% block content %}
        
        {% endblock %}
    </body>
</html>

In [ ]:
# home.html
{% extends "base.html" %}
{% block content %}
    <div class = "jumbotron">
        {% if current_user.is_authenticated %}
        <p>Hi {{current_user.username}}</p>
        {% else %}
        <p>Please login or register</p>
        {% endif %}
    </div>
{% endblock %}

In [ ]:
# welcome_user.html
{% extends "base.html" %}
{% block content %}
    <div class = "jumbotron">
        <p> congrats! You are logged in! </p> 
    </div>
{% endblock %}

In [ ]:
# login.html 
{% extends "base.html" %}
{% block content %}
    <form method = "post">
        {{form.hidden_tag()}}
        {{form.email.label}}{{form.email()}}
        {{form.password.label}}{{form.password()}}
        {{form.submit()}}
    </form>
{% endblock %}

In [ ]:
# register.html
{% extends "base.html" %}
{% block content %}
    <form method = "post">
        {{form.hidden_tag()}}
        {{form.email.label}}{{form.email()}}<br>
        {{form.username.label}}{{form.username()}}<br>
        {{form.password.label}}{{form.password()}}<br>
        {{form.password_confirm.label}}{{form.pass_confirm}}<br>
        {{form.submit()}}
    </form>
{% endblock %}

Source Jose Portilla 

This example is large, and can be a lot to take in, but, it is a very good way to see how all the things we have learned about using HTML, Bootstrap, and Flask can tie together to make a simple yet effective website!!!

## Flask OAuth

Many sites do not want to deal with the responsibility of handling users and their profiles and the authorization of them, so they use services that are used as logins

You have likely seen this when you are trying to login or register an account with something, and it gives you the option to __"login with google"__ or __"login with facebook"__ or something along those lines 

The __Flask-Dance__ library is what we use to add these OAuth backend features 

![SegmentLocal](https://media3.giphy.com/media/6s1hcLp1VQzTSRu7Jl/giphy.gif "segment")

An important part about this feature is that while it allows you to use the login authorization but does not give access to the assets of that login

So in the google example, it would give you the google login, but would not give access to their google account 

In [ ]:
## To get Flask-Dance!!:
pip install Flask-Dance

To use Flask-Dance, it is best to look at the documetation, which can be found at : __flask-dance.readthedocs.io__

This has all the information you need, with quickstarts to the different services, including Google QuickStart, Twitter QuickStart, and GitHub QuickStart. These tell you how to set up the application itself, as well as providing the basic code to set up the OAuth 